In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import boto3
from botocore.exceptions import NoCredentialsError
import uuid
import os

app = Flask(__name__)
CORS(app, resources={r"/": {"origins": "*"}}, supports_credentials=True)

s3 = boto3.client(
    's3',
    region_name='us-east-1',
    aws_access_key_id='add yours',
    aws_secret_access_key='add yours'
)

BUCKET_NAME = 'omnia-shahd'

def get_file_folder(filename):
    extension = os.path.splitext(filename)[1].lower()
    
    if extension in ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp']:
        return 'images/'
    elif extension in ['.pdf', '.doc', '.docx', '.xls', '.xlsx', '.ppt', '.pptx', '.txt']:
        return 'documents/'
    elif extension in ['.mp4', '.mov', '.avi', '.mkv', '.wmv']:
        return 'videos/'
    else:
        return 'others/'

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']

    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    try:
        folder = get_file_folder(file.filename)
        filename = f"{folder}{uuid.uuid4().hex}_{file.filename}"
        
        s3.upload_fileobj(file, BUCKET_NAME, filename)

        file_url = f"https://{BUCKET_NAME}.s3.amazonaws.com/{filename}"

        return jsonify({
            'message': 'File uploaded successfully',
            'url': file_url,
            'folder': folder[:-1]  
        }), 200

    except NoCredentialsError:
        return jsonify({'error': 'Credentials not available'}), 500
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/files', methods=['GET'])
def list_files():
    try:
        response = s3.list_objects_v2(Bucket=BUCKET_NAME)

        if 'Contents' not in response:
            return jsonify({'files': []}), 200

        files = []
        for obj in response['Contents']:
            file_url = f"https://{BUCKET_NAME}.s3.amazonaws.com/{obj['Key']}"
            files.append({
                'url': file_url,
                'name': obj['Key'].split('/')[-1],
                'folder': obj['Key'].split('/')[0]
            })

        return jsonify({'files': files}), 200

    except Exception as e:
        return jsonify({'error': str(e)}), 500


In [2]:
if __name__ == '__main__':
    app.run(port=5000, debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Jun/2025 07:23:27] "POST /upload HTTP/1.1" 200 -
